In [1]:
## Task: 
## (1) Which keywords can let your comments get a higher score in Reddit? 
## 2. Compared the run time of (1) in common way and pyspark
import time
import re
import pandas as pd
from pyspark.sql import SparkSession
from operator import add
import os
from pyspark.sql.functions import length
from Pre_processing import data  # import data from Pre_processing

In [2]:
# preparation for Task(1) and (2)
start_time = time.time()
stop_words = ['a', 'an', 'and', 'are', 'as', 'at', 'be', 'but', 'by', 'for', 'if', 'in', 'into', 'is', 'it',
              'no', 'not', 'of', 'on', 'or', 'such', 'that', 'the', 'their', 'then', 'there', 'these', 'they',
              'this', 'to', 'was', 'will', 'with', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '-', 
              '_', '+', '=', '[', ']', '{', '}', ';', ':', '"', "'", '<', '>', ',', '.', '/', '?', '\\', 
              '|', '`', '~', '']

spark_session = SparkSession.builder\
.master("spark://192.168.2.70:7077") \
.appName("yufengchen_app")\
.config("spark.dynamicAllocation.enabled", True)\
.config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
.config("spark.shuffle.service.enabled", True)\
.config("spark.dynamicAllocation.executorIdleTimeout","30s")\
.config("spark.cores.max", 4)\
.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/02 12:04:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.functions import length, split, explode, sum, count, avg, col


def use_pyspark_function(data, spark_session=spark_session):
    word_score = {}
    word_count = {}
    spark_context = spark_session.sparkContext
    spark_context.setLogLevel("ERROR")  
    data_pyspark = spark_session.createDataFrame(data)
    print('Using Pyspark')
    start_time = time.time()                                                                                           
    data_pyspark = data_pyspark.select('score', 'body').filter((length('body') >= 10))

    
    """
    word_score = data_pyspark.rdd.flatMap(lambda row: [(word, row['score']) for word in row['body'].split() if word not in stop_words]) \
        .reduceByKey(lambda a, b: a + b) \
        .collectAsMap()
    word_count = data_pyspark.rdd.flatMap(lambda row: [(word, 1) for word in row['body'].split() if word not in stop_words]) \
        .reduceByKey(lambda a, b: a + b) \
        .collectAsMap()


    # filter the word appear less than 10 times
    filtered_word_count = {k: v for k, v in word_count.items() if v >= 10}
    filtered_word_score = {k: v for k, v in word_score.items() if k in filtered_word_count.keys()}


    for word,times in filtered_word_count.items():
        if filtered_word_score[word] != 0:
            filtered_word_score[word] = filtered_word_score[word]/times
            
   
    sorted_filtered_word_score = dict(sorted(filtered_word_score.items(), key=lambda x: x[1], reverse=True))
    # top 10 word  that may let your comments get a higher score in Reddit
    for key, value in list(sorted_filtered_word_score.items())[:10]:
        print(f"{key}: {value}")
    """  
    
    broadcast_stop_words = spark_context.broadcast(stop_words)

    # Compute the word score and count RDDs
    word_score_rdd = data_pyspark.rdd.flatMap(
        lambda row: [(word, row['score']) for word in row['body'].split() if word not in broadcast_stop_words.value]
    ).reduceByKey(lambda a, b: a + b)

    word_count_rdd = data_pyspark.rdd.flatMap(
        lambda row: [(word, 1) for word in row['body'].split() if word not in broadcast_stop_words.value]
    ).reduceByKey(lambda a, b: a + b)

    # Filter out words that appear less than 10 times
    filtered_word_count_rdd = word_count_rdd.filter(lambda x: x[1] >= 10).cache()
    filtered_word_score_rdd = word_score_rdd.join(filtered_word_count_rdd).mapValues(lambda x: x[0] / x[1])

    # Sort the results by score and take the top 10
    top_10_words_rdd = filtered_word_score_rdd.sortBy(lambda x: x[1], ascending=False).take(10)

    # Print the results
    for word, score in top_10_words_rdd:
        print(f"{word}: {score}")
    end_time = time.time()
    print('Spend:', end_time-start_time,'s')


In [4]:
def without_pyspark_function(data):
    word_score = {}
    word_count = {}
    print('Without using Pyspark')
    start_time = time.time()
    data = data.loc[:, ['score', 'body']]
    data = data[data['body'].str.len() >= 10] # drop len(body) less than 10 
    
    for index, row in data.iterrows():
        score = row['score']
        words_list = row['body'].split()
        for word in words_list:
            # filter stop word
            if word not in stop_words:
                if word not in word_count.keys():
                    word_count[word] = 1
                    word_score[word] = score
                else:
                    word_count[word] = 1 + word_count[word]
                    word_score[word] = score + word_score[word]

    filtered_word_count = {k: v for k, v in word_count.items() if v >= 10}
    filtered_word_score = {k: v for k, v in word_score.items() if k in filtered_word_count.keys()}

    for word,times in filtered_word_count.items():
        if filtered_word_score[word] != 0:
            filtered_word_score[word] = filtered_word_score[word]/times
        else:
            pass    

    sorted_filtered_word_score = dict(sorted(filtered_word_score.items(), key=lambda x: x[1], reverse=True))
    # top 10 word  that may let your comments get a higher score in Reddit
    for key, value in list(sorted_filtered_word_score.items())[:10]:
        print(f"{key}: {value}")

    end_time = time.time()
    print('Spend:', end_time-start_time,'s')

In [5]:
without_pyspark_function(data)

Without using Pyspark
fantastic,: 80.81818181818181
famous.: 80.2
a:: 68.2
fundamentalism,: 62.45454545454545
times): 58.25
liquids: 56.07142857142857
analytical: 47.05882352941177
===========================: 47.0
vocational: 46.80769230769231
creationism.: 45.86666666666667
Spend: 9.641444683074951 s


In [6]:
use_pyspark_function(data)

/home/ubuntu/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Using Pyspark


fantastic,: 80.81818181818181
famous.: 80.2
a:: 68.2
fundamentalism,: 62.45454545454545
times): 58.25
liquids: 56.07142857142857
analytical: 47.05882352941177
===========================: 47.0
vocational: 46.80769230769231
creationism.: 45.86666666666667
Spend: 15.22739839553833 s
